SIGN LANGUAGE RECOGNITION

Group 7 :

Anar Shikhaliyev, Asmar Khalilli, Sabina Abdyeva, Vugar Mammadov


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam

In [3]:
train_path = './Dataset/train_set'
test_path = './Dataset/test_set'
classes = 4 # number of classes, 26 in normal case, but we trained on 4 classes 
img_size = 64 # 64x64 pixels

train_batches = ImageDataGenerator (
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    horizontal_flip=False,
    fill_mode='nearest' ).flow_from_directory (
        train_path,
        target_size=(img_size, img_size),
        batch_size=23,
        color_mode = 'grayscale',
        class_mode='categorical',
        shuffle=True )

test_batches = ImageDataGenerator(
    rescale=1./255 ).flow_from_directory (
        test_path,
        target_size=(img_size, img_size),
        batch_size=32,
        color_mode = 'grayscale',
        class_mode='categorical',
        shuffle=True )

Found 45500 images belonging to 26 classes.
Found 6500 images belonging to 26 classes.


In [ ]:
model = Sequential()

# 1st Convolution layer
model.add(Conv2D(32, kernel_size=(3,3), activation = 'relu', input_shape=(img_size, img_size ,1) ))
# Pooling
model.add(MaxPooling2D(pool_size = (2, 2)))

# 2nd Convolution layer
model.add(Conv2D(64, kernel_size = (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

# 3rd Convolution layer
model.add(Conv2D(64, kernel_size = (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

# Flattening
model.add(Flatten())

# Full connected layer
model.add(Dense(256, activation = 'relu'))
model.add(Dropout(0.50))

model.add(Dense(classes, activation = 'softmax'))
print(model.summary())

In [ ]:
model.compile (
    loss = 'categorical_crossentropy',
    optimizer= Adam(learning_rate=0.0005),
    metrics=['accuracy'] )

In [ ]:
nb_train_samples = 7000 # 45500 : here we should write the number of training and testing data
                                # this numbers are given as the output of the second cell
nb_test_samples = 1000 # 6500 
epochs = 10
batch_size = 32

history = model.fit (
    train_batches,
    steps_per_epoch = nb_train_samples // batch_size,
    validation_data = test_batches,
    validation_steps = nb_test_samples // batch_size,
    epochs = epochs )

In [ ]:
# Graph of accuracies of both train and test datasets
plt.title("Accuracy")
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.xlabel('Number of Epoches')
plt.ylabel('Accuracy')
plt.legend(['Train data','Test data'])

plt.show()

In [ ]:
# Save the model 
model.save("gestures_cnn.h5")
model = load_model('gestures_cnn.h5')

In [ ]:
def getLetter(result):
    labels = { 
        0: 'A',
        1: 'B',
        2: 'C',
        3: 'D',
        4: 'E',
        5: 'F',
        6: 'G',
        7: 'H',
        8: 'I',
        9: 'J',
        10: 'K',
        11: 'L',
        12: 'M',
        13: 'N',
        14: 'O',
        15: 'P',
        16: 'Q',
        17: 'R',
        18: 'S',
        19: 'T',
        20: 'U',
        21: 'V',
        22: 'W',
        23: 'X',
        24: 'Y',
        25: 'Z' }
    try:
        res = int(result)
        return labels[res]
    except:
        return "Something went wrong :/"

In [ ]:
cap = cv2.VideoCapture(0)

while True:

    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)
    frame_copy = frame.copy()

    roi = frame[100:300, 350:550]
    roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    roi = cv2.resize(roi, (img_size, img_size), interpolation = cv2.INTER_AREA)
    cv2.imshow('roi - gray', roi)

    cv2.rectangle(frame_copy, (350, 100), (550, 300), (255,0,0), 5)
    
    roi = roi.reshape(1,img_size,img_size,1) 

    roi = roi/255
    result = str(np.argmax(model.predict(roi, 1, verbose = 0)[0]))

    cv2.putText(frame_copy, getLetter(result), (300 , 100), cv2.FONT_HERSHEY_COMPLEX, 2, (0, 255, 0), 2)
    cv2.imshow('Frame', frame_copy)    
    
    if cv2.waitKey(1) == 13: # Enter-Key to stop
        break
        
cap.release()
cv2.destroyAllWindows() 